<a href="https://colab.research.google.com/github/Flychuban/Stocks-Crypto-Research/blob/main/Stocks_Crypto_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [3]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name) 

In [ ]:
url = "https://uk.finance.yahoo.com/news/d-put-2-000-tesla-043029225.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [ ]:
paragraphs

In [ ]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [ ]:
ARTICLE

'RCB’s owners have focused on Virat Kohli to leverage their commercial brand Both Tesla (NASDAQ: TSLA) and NIO (NYSE: NIO) stock declined by more than 50% in value in 2022. It was a dreadful year for most growth shares, including electric vehicle (EV) companies. But what if I’d taken a contrarian stance and decided to invest £1,000 in each of these fallen stocks as a New Year gift for myself? How much would I have today? Well, Tesla shares are up a very impressive 65% so far this year. In contrast, NIO shares have declined 17% since the end of December and now sit at just under $8 per share. This means that my Tesla holding would be worth £1,650, while the value of my position in its Chinese EV rival would have fallen to £830. So, my overall investment would be worth £2,480 today. That’s a gain of 24%, which is an exceptional return after just a few months. But what about the future? Should I buy either or both stocks today? There seem to be two big reasons why Tesla stock has come bac

In [ ]:
input_ids = tokenizer.encode(ARTICLE, return_tensors = 'pt')
output = model.generate(input_ids, max_length = 55, num_beams = 5, early_stopping = True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
summary

'Tesla stock is up 65% so far this year, while NIO shares are down 17%.'

In [4]:
monitored_tickers = ['BTC', 'TSLA', 'ETH']

In [5]:
def search_stock_news_urls(ticker):
  search_url = f'https://www.google.com/search?q=yahoo+finance+{ticker}&tbm=nws'
  r = requests.get(search_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all('a')
  hrefs = [link['href'] for link in atags]
  return hrefs

In [6]:
raw_urls = {ticker: search_stock_news_urls(ticker) for ticker in monitored_tickers}

In [ ]:
raw_urls

In [7]:
import re

In [9]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [10]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [ ]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

In [12]:
def scrape_and_process(URLs):
  ARTICLES = []
  for url in URLs:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = [paragraph.text for paragraph in paragraphs]
    words = ' '.join(text).split(' ')[:350]
    ARTICLE = ' '.join(words)
    ARTICLES.append(ARTICLE)
  return ARTICLES

In [ ]:
articles = {ticker: scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

In [14]:
def summarize(articles):
  summaries = []
  for article in articles:
    input_ids = tokenizer.encode(article, return_tensors = 'pt')
    output = model.generate(input_ids, max_length = 55, num_beams = 5, early_stopping = True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    summaries.append(summary)
  return summaries

In [ ]:
summaries = {ticker: summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [17]:
MODEL_NAME = 'RashidNLP/Finance_Multi_Sentiment'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bert_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = 3).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [46]:
def get_sentiment(sentences):
    output = []
    bert_dict = {}
    vectors = tokenizer(sentences, padding = True, max_length = 65, return_tensors='pt').to(device)
    outputs = bert_model(**vectors).logits
    probs = torch.nn.functional.softmax(outputs, dim = 1)
    for prob in probs:
        bert_dict['neg'] = round(prob[0].item(), 3)
        bert_dict['neu'] = round(prob[1].item(), 3)
        bert_dict['pos'] = round(prob[2].item(), 3)
        results = []
        results.append(bert_dict['neg'])
        results.append(bert_dict['neu'])
        results.append(bert_dict['pos'])    
        max_value = results[0]
        label = 'NEGATIVE'
        if results[1] > max_value and results[1] > results[2]:
          max_value = results[1]
          label = 'NEUTRAL'
        elif results[1] > max_value and not results[1] > results[2]:
          max_value = results[2]
          label = 'POSITIVE'
        elif not results[1] > max_value and results[2] > max_value:
          max_value = results[2]
          label = 'POSITIVE'
        
        output.append([label, max_value])
    return output

In [ ]:
scores = {ticker: get_sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

In [50]:
def create_output_array(summaries, scores, cleaned_urls):
  output = []
  for ticker in monitored_tickers:
    for counter in range(len(summaries[ticker])):
      output_this = [
          ticker,
          summaries[ticker][counter],
          scores[ticker][counter][0], # this is for LABEL
          scores[ticker][counter][1],  # this is for PERCENTAGE
          cleaned_urls[ticker][counter]
      ]
      output.append(output_this)
  return output

In [53]:
final_output = create_output_array(summaries, scores, cleaned_urls)

In [54]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Score', 'URL'])

In [ ]:
final_output

In [59]:
import csv

with open("summary.csv", mode='w', newline='') as file:
  csv_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  csv_writer.writerows(final_output)